In [1]:
import os, sys, time, importlib
sys.path.append('/home/public/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import Network_Clean as gnClean
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx
from shapely.geometry import Point
import numpy as np

### Read cleaned graph object

In [5]:
pth_base = '/home/wb514197/data/dhaka'
gostNetsPickle = '/home/wb514197/data/dhaka/dhaka_clean_small.pickle'

In [6]:
if not os.path.exists(gostNetsPickle):
    G_losm = losm.OSM_to_network(inRoads)
    G_losm.generateRoadsGDF(verbose = False)
    G_losm.initialReadIn()       
    G = gnClean.CleanNetwork(G_losm.network, junctdist = 0.1)
    #Isolate the largest sub-graph
    g_small = gnClean.largestSubgraph(GC)
    gn.save(g_small, "dhaka_clean_small", gostNetsFolder)    
else:
    G = nx.read_gpickle(gostNetsPickle)

In [160]:
# No need to salt, as we are using the entire network
#G = gn.salt_long_lines(G, thresh=2000, factor=1000, source='epsg:4326', target='epsg:32646')
G_largest_time = gn.convert_network_to_time(G, distance_tag = 'length', default='secondary', 
                                            road_col='infra_type', factor=1000)

### Read edges with traffic travel times

In [161]:
edges_duration = pd.read_csv(os.path.join(pth_base,'mac/edges_duration.csv'))

In [163]:
edges_duration['newID'] = edges_duration['stnode'].astype(str)+"_"+edges_duration['endnode'].astype(str)+"_"+edges_duration['id'].astype(str)
edges_duration = edges_duration[['newID','duration']]
edges_duration = edges_duration.set_index('newID')

In [164]:
edges_duration.head()

,duration
newID,
8_13909_9633,21.0
8_17612_9660,65.6
11_3526_11084,13.6
11_8990_11083,21.4
18_2317_4630,8.1


In [165]:
len(edges_duration)

11028

In [166]:
edges_duration = edges_duration.dropna()

In [167]:
len(edges_duration)

11026

In [168]:
edges_duration.loc[edges_duration.index.duplicated()]

,duration
newID,


In [169]:
for u, v, data in G_largest_time.edges(data = True):
    data['newID'] = str(u)+"_"+str(v)+"_"+str(data['id'])
    if data['newID'] in edges_duration.index:
        data['time_new'] = edges_duration.loc[data['newID']][0]
        data['mapbox_api'] = 1
    else:
        data['time_new'] = data['time']
        data['mapbox_api'] = 0

In [170]:
gn.example_edge(G_largest_time)

(0, 4244, {'Wkt': 'LINESTRING (90.35207870000001 23.7962001, 90.35269099999999 23.7962498)', 'id': 22077, 'infra_type': 'residential', 'osm_id': '466261136', 'key': 'edge_22077', 'length': 62.642636201901624, 'Type': 'legitimate', 'time': 11.275674516342292, 'mode': 'drive', 'newID': '0_4244_22077', 'time_new': 11.275674516342292, 'mapbox_api': 0})


In [171]:
new_edges = gn.edge_gdf_from_graph(G_largest_time)

In [172]:
edges_edited = new_edges[new_edges.mapbox_api==1]

In [173]:
edges_edited.head(100)

,stnode,endnode,id,time_new,time,length,infra_type,mapbox_api,osm_id,Type,newID,Wkt,key,mode,geometry
16,8,13909,9633,21.0,11.182156,93.184636,tertiary,1,239878634,legitimate,8_13909_9633,"LINESTRING (90.40712190000001 23.7182804, 90.4...",edge_9633,drive,"LINESTRING (90.40712190000001 23.7182804, 90.4..."
17,8,17612,9660,65.6,31.339852,261.165435,tertiary,1,240115343,legitimate,8_17612_9660,"LINESTRING (90.4060934 23.7203881, 90.40644469...",edge_9660,drive,"LINESTRING (90.40712190000001 23.7182804, 90.4..."
26,11,3526,11084,13.6,5.449893,45.415774,tertiary,1,306064948,legitimate,11_3526_11084,"LINESTRING (90.38636990000001 23.8845511, 90.3...",edge_11084,drive,"LINESTRING (90.38636990000001 23.8845511, 90.3..."
27,11,8990,11083,21.4,9.304712,77.539270,tertiary,1,306064948,legitimate,11_8990_11083,"LINESTRING (90.38646009999999 23.885246, 90.38...",edge_11083,drive,"LINESTRING (90.38636990000001 23.8845511, 90.3..."
47,18,2317,4630,8.1,4.289768,35.748065,tertiary,1,33798206,legitimate,18_2317_4630,"LINESTRING (90.36177929999999 23.7450579, 90.3...",edge_4630,drive,"LINESTRING (90.36177929999999 23.7450579, 90.3..."
49,18,10325,4629,10.4,3.518653,29.322110,tertiary,1,33798206,legitimate,18_10325_4629,"LINESTRING (90.36149880000001 23.7449994, 90.3...",edge_4629,drive,"LINESTRING (90.36177929999999 23.7450579, 90.3..."
57,22,19508,318,33.0,7.396466,82.182956,primary,1,13844837,legitimate,22_19508_318,"LINESTRING (90.42567080000001 23.7805355, 90.4...",edge_318,drive,"LINESTRING (90.42567080000001 23.7805355, 90.4..."
58,22,26161,33707,772.3,27.474813,305.275697,primary,1,566624890,legitimate,22_26161_33707,"LINESTRING (90.4259091 23.7832826, 90.4258087 ...",NaN,drive,"LINESTRING (90.4259091 23.7832826, 90.42567459..."
70,27,486,35258,57.2,22.530551,187.754591,tertiary,1,610021906,legitimate,27_486_35258,"LINESTRING (90.3724933 23.837617, 90.3733194 2...",edge_35258,drive,"LINESTRING (90.37433009999999 23.8377161, 90.3..."
71,27,1471,35259,10.8,1.389003,11.575027,tertiary,1,610021907,legitimate,27_1471_35259,"LINESTRING (90.37433009999999 23.8377161, 90.3...",edge_35259,drive,"LINESTRING (90.37433009999999 23.8377161, 90.3..."


In [268]:
edges_edited.to_file(os.path.join(gostNetsFolder, "edges_edited.shp"), driver='ESRI Shapefile')

### Load origins and destinations, get unique origin nodes before running OD

In [174]:
origins_pop = gpd.read_file(os.path.join(pth_base, 'Dhaka_pop_2015.shp'))
dest_centroids = gpd.read_file(os.path.join(pth_base, 'Dhaka_Upzilla_centroids_edit.shp'))
origins_pop_snapped = gn.pandana_snap(G_largest_time, origins_pop, source_crs = 'epsg:4326', target_crs = 'epsg:32646', add_dist_to_node_col=True)
dest_centroids_snapped = gn.pandana_snap(G_largest_time, dest_centroids, source_crs = 'epsg:4326', target_crs = 'epsg:32646', add_dist_to_node_col=False)
origins_pop_nodes = list(set(origins_pop_snapped.NN))
dest_centroids_nodes = list(dest_centroids_snapped.NN)

/home/public/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1676: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())
/home/public/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1678: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(in_df[['x','y']].as_matrix())
/home/public/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1687: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())
/home/public/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1689: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(in_df[['x','y']].as_matrix())


In [175]:
od_time = gn.calculate_OD(G_largest_time, origins=origins_pop_nodes, 
                          destinations=dest_centroids_nodes, fail_value=99999999, weight='time_new')

In [182]:
od_time[od_time==99999999]

array([], dtype=float64)

In [176]:
od_time_df = pd.DataFrame(od_time, index=origins_pop_nodes, columns=dest_centroids_snapped.ORIG_FID)
od_time_df.head()

ORIG_FID,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
3,453.384222,2567.072424,2065.009773,3007.466757,3130.776817,1772.321674,4175.692394,1092.439367,3822.476097,786.421905,...,1525.109991,1628.727826,517.218384,2915.771556,2383.935111,995.673494,1492.620035,2852.831461,4174.630093,3336.271295
4,2166.585124,1450.501861,1664.088878,2860.402791,3610.010994,2004.304700,3829.051381,2790.230589,1643.694170,1727.491206,...,2941.911540,1945.556673,2048.501126,1966.003283,1526.342004,1157.043534,1028.696906,4304.273412,3827.989080,3429.940572
7,1699.655336,1485.884178,2361.255748,2702.692226,2826.002287,2108.453716,3767.912317,2006.163398,2706.619093,1260.561418,...,2157.844349,1849.761153,1281.206764,3018.897374,2579.236094,616.031051,157.523157,3545.904398,3766.850016,3031.496764
8,2173.268222,3047.768018,81.269094,4457.668948,4704.917110,413.072917,5426.317539,2853.206568,1980.119492,1347.052649,...,3422.695212,622.055464,2414.803605,868.277404,330.813979,1856.442128,2003.864529,4741.007604,5425.255237,4910.411588
10,3359.999270,3259.022046,1226.000142,4668.922975,5169.573863,1553.378388,5637.571566,4039.937617,1191.510891,2533.783697,...,4344.810684,1808.786512,3451.400269,917.827525,1016.788179,2559.942677,2431.596049,5732.870732,5636.509265,5238.460757


Add walking time (from origin to NN) for each OD

In [271]:
# origins_join = origins_pop_snapped.merge(od_time_df, how='left', on='NN')
walk_speed = 4.5 # km/h
origins_pop_snapped['NN_dist_seconds'] = ((origins_pop_snapped.NN_dist / 1000) / walk_speed) * 60 * 60
origins_join = origins_pop_snapped.join(od_time_df, on='NN', rsuffix="dist_")

In [272]:
origins_join.head()

,pointid,pop,geometry,NN,NN_dist,NN_dist_seconds,0,1,2,3,...,31,32,33,34,35,36,37,38,39,40
0,1,68.5005,POINT (90.37997478300007 23.89754883000005),23968,80.900010,64.720008,2898.874367,2725.742511,5006.239245,1323.344207,...,2361.949978,4569.957299,3275.765314,5735.315527,5295.654248,3256.455608,3417.374566,1183.108445,1752.233598,750.609474
1,2,73.0887,POINT (90.38080808300006 23.89754883000005),23968,17.384936,13.907949,2898.874367,2725.742511,5006.239245,1323.344207,...,2361.949978,4569.957299,3275.765314,5735.315527,5295.654248,3256.455608,3417.374566,1183.108445,1752.233598,750.609474
2,3,79.5437,POINT (90.38164138300007 23.89754883000005),25270,22.985724,18.388579,2923.574367,2689.642511,5030.939245,1287.244207,...,2386.649978,4594.657299,3300.465314,5699.215527,5259.554248,3220.355608,3381.274566,1207.808445,1716.133598,714.509474
3,4,72.5300,POINT (90.38247468300007 23.89754883000005),25270,63.769252,51.015402,2923.574367,2689.642511,5030.939245,1287.244207,...,2386.649978,4594.657299,3300.465314,5699.215527,5259.554248,3220.355608,3381.274566,1207.808445,1716.133598,714.509474
4,5,71.7939,POINT (90.37997478300007 23.89671553000005),12327,84.453865,67.563092,2866.474367,2762.542511,4973.839245,1360.144207,...,2329.549978,4537.557299,3243.365314,5769.486628,5292.764583,3290.626709,3451.545668,1150.708445,1789.033598,787.409474


In [228]:
origins_join.columns[6:len(origins_join.columns)]

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40],
      dtype='object')

In [273]:
od_time_df_adjusted = origins_join.iloc[:,6:len(origins_join.columns)].apply(lambda x: x+origins_join['NN_dist_seconds'], axis=0)

In [274]:
od_time_df_adjusted.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,2963.594375,2790.462519,5070.959253,1388.064215,2199.096531,4778.271154,2259.746261,2356.663369,5487.757254,3792.371385,...,2426.669986,4634.677307,3340.485322,5800.035535,5360.374255,3321.175616,3482.094574,1247.828453,1816.953606,815.329482
1,2912.782316,2739.650460,5020.147194,1337.252156,2148.284472,4727.459096,2208.934202,2305.851310,5436.945195,3741.559327,...,2375.857927,4583.865248,3289.673263,5749.223476,5309.562197,3270.363557,3431.282515,1197.016394,1766.141547,764.517423
2,2941.962946,2708.031090,5049.327824,1305.632786,2116.665102,4756.639725,2177.314832,2335.031940,5405.325825,3770.739956,...,2405.038557,4613.045878,3318.853893,5717.604106,5277.942826,3238.744187,3399.663145,1226.197024,1734.522177,732.898053
3,2974.589768,2740.657913,5081.954647,1338.259608,2149.291925,4789.266548,2209.941655,2367.658763,5437.952648,3803.366779,...,2437.665380,4645.672701,3351.480716,5750.230929,5310.569649,3271.371009,3432.289968,1258.823847,1767.149000,765.524875
4,2934.037459,2830.105603,5041.402337,1427.707299,2238.739615,4748.714238,2299.389345,2327.106453,5524.771439,3762.814469,...,2397.113070,4605.120391,3310.928406,5837.049720,5360.327675,3358.189801,3519.108759,1218.271537,1856.596690,854.972566


In [275]:
od_time_df_adjusted.shape

(28165, 41)

### Calculate accessible jobs within 15, 30, 45, and 60 minutes

In [306]:
combinedRes = []
question = 'Q17_Male'
maleJobs = dest_centroids[question]    
for minute_cut in [15,30,45,60]:
    allRes = []
    for idx in range(0, od_time_df_adjusted.shape[0]):
        x = od_time_df_adjusted.iloc[idx]
        x[x==-1] = 99999
        allRes.append(sum(maleJobs[x < (minute_cut*60)]))
    combinedRes.append(allRes)
output = pd.DataFrame(combinedRes).transpose()
output.columns = ['nMales15','nMales30','nMales45','nMales60']
output['idx'] = od_time_df_adjusted.index
origins_pop_snapped['idx'] = origins_pop_snapped.index
origins_results = origins_pop_snapped.merge(output, how='left', on='idx')
origins_results.to_csv(os.path.join(gostNetsFolder, "%s_jobs_v3.csv" % question))

In [308]:
combinedRes = []
question = 'Q17_Female'
maleJobs = dest_centroids[question]    
for minute_cut in [15,30,45,60]:
    allRes = []
    for idx in range(0, od_time_df_adjusted.shape[0]):
        x = od_time_df_adjusted.iloc[idx]    
        x[x==-1] = 99999
        allRes.append(sum(maleJobs[x < (minute_cut*60)]))
    combinedRes.append(allRes)
output = pd.DataFrame(combinedRes).transpose()
output.columns = ['nFemales15','nFemales30','nFemales45','nFemales60']
output['idx'] = od_time_df_adjusted.index
origins_pop_snapped['idx'] = origins_pop_snapped.index
origins_results = origins_pop_snapped.merge(output, how='left', on='idx')
origins_results.to_csv(os.path.join(gostNetsFolder, "%s_jobs_v3.csv" % question))